# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fee0d67aa60>
├── 'a' --> tensor([[-0.9584,  1.1344,  0.1895],
│                   [-0.4369, -2.1558, -1.7903]])
└── 'x' --> <FastTreeValue 0x7fee085fc940>
    └── 'c' --> tensor([[-0.8380, -0.9573,  0.9886, -1.1453],
                        [-0.2568, -0.1171,  0.4791,  0.4327],
                        [ 0.2909, -0.2474,  0.5359,  0.8001]])

In [4]:
t.a

tensor([[-0.9584,  1.1344,  0.1895],
        [-0.4369, -2.1558, -1.7903]])

In [5]:
%timeit t.a

86 ns ± 0.16 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fee0d67aa60>
├── 'a' --> tensor([[ 0.0491,  0.1547,  0.5780],
│                   [-0.6319,  1.0889, -0.0507]])
└── 'x' --> <FastTreeValue 0x7fee085fc940>
    └── 'c' --> tensor([[-0.8380, -0.9573,  0.9886, -1.1453],
                        [-0.2568, -0.1171,  0.4791,  0.4327],
                        [ 0.2909, -0.2474,  0.5359,  0.8001]])

In [7]:
%timeit t.a = new_value

83.9 ns ± 0.853 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.9584,  1.1344,  0.1895],
               [-0.4369, -2.1558, -1.7903]]),
    x: Batch(
           c: tensor([[-0.8380, -0.9573,  0.9886, -1.1453],
                      [-0.2568, -0.1171,  0.4791,  0.4327],
                      [ 0.2909, -0.2474,  0.5359,  0.8001]]),
       ),
)

In [10]:
b.a

tensor([[-0.9584,  1.1344,  0.1895],
        [-0.4369, -2.1558, -1.7903]])

In [11]:
%timeit b.a

76.3 ns ± 0.638 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.9439,  0.3156,  0.7804],
               [ 0.6786, -0.9633,  0.2760]]),
    x: Batch(
           c: tensor([[-0.8380, -0.9573,  0.9886, -1.1453],
                      [-0.2568, -0.1171,  0.4791,  0.4327],
                      [ 0.2909, -0.2474,  0.5359,  0.8001]]),
       ),
)

In [13]:
%timeit b.a = new_value

638 ns ± 3.23 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.06 µs ± 7.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.6 µs ± 98.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

166 µs ± 1.86 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

167 µs ± 1.41 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fee0d63e100>
├── 'a' --> tensor([[[-0.9584,  1.1344,  0.1895],
│                    [-0.4369, -2.1558, -1.7903]],
│           
│                   [[-0.9584,  1.1344,  0.1895],
│                    [-0.4369, -2.1558, -1.7903]],
│           
│                   [[-0.9584,  1.1344,  0.1895],
│                    [-0.4369, -2.1558, -1.7903]],
│           
│                   [[-0.9584,  1.1344,  0.1895],
│                    [-0.4369, -2.1558, -1.7903]],
│           
│                   [[-0.9584,  1.1344,  0.1895],
│                    [-0.4369, -2.1558, -1.7903]],
│           
│                   [[-0.9584,  1.1344,  0.1895],
│                    [-0.4369, -2.1558, -1.7903]],
│           
│                   [[-0.9584,  1.1344,  0.1895],
│                    [-0.4369, -2.1558, -1.7903]],
│           
│                   [[-0.9584,  1.1344,  0.1895],
│                    [-0.4369, -2.1558, -1.7903]]])
└── 'x' --> <FastTreeValue 0x7fed3d817910>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

37.9 µs ± 283 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fed3d7a7880>
├── 'a' --> tensor([[-0.9584,  1.1344,  0.1895],
│                   [-0.4369, -2.1558, -1.7903],
│                   [-0.9584,  1.1344,  0.1895],
│                   [-0.4369, -2.1558, -1.7903],
│                   [-0.9584,  1.1344,  0.1895],
│                   [-0.4369, -2.1558, -1.7903],
│                   [-0.9584,  1.1344,  0.1895],
│                   [-0.4369, -2.1558, -1.7903],
│                   [-0.9584,  1.1344,  0.1895],
│                   [-0.4369, -2.1558, -1.7903],
│                   [-0.9584,  1.1344,  0.1895],
│                   [-0.4369, -2.1558, -1.7903],
│                   [-0.9584,  1.1344,  0.1895],
│                   [-0.4369, -2.1558, -1.7903],
│                   [-0.9584,  1.1344,  0.1895],
│                   [-0.4369, -2.1558, -1.7903]])
└── 'x' --> <FastTreeValue 0x7fed3d7a7a60>
    └── 'c' --> tensor([[-0.8380, -0.9573,  0.9886, -1.1453],
                        [-0.2568, -0.1171,  0.4791,  0.4327],
                 

In [23]:
%timeit t_cat(trees)

36.3 µs ± 245 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

78.9 µs ± 658 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.9584,  1.1344,  0.1895],
                [-0.4369, -2.1558, -1.7903]],
       
               [[-0.9584,  1.1344,  0.1895],
                [-0.4369, -2.1558, -1.7903]],
       
               [[-0.9584,  1.1344,  0.1895],
                [-0.4369, -2.1558, -1.7903]],
       
               [[-0.9584,  1.1344,  0.1895],
                [-0.4369, -2.1558, -1.7903]],
       
               [[-0.9584,  1.1344,  0.1895],
                [-0.4369, -2.1558, -1.7903]],
       
               [[-0.9584,  1.1344,  0.1895],
                [-0.4369, -2.1558, -1.7903]],
       
               [[-0.9584,  1.1344,  0.1895],
                [-0.4369, -2.1558, -1.7903]],
       
               [[-0.9584,  1.1344,  0.1895],
                [-0.4369, -2.1558, -1.7903]]]),
    x: Batch(
           c: tensor([[[-0.8380, -0.9573,  0.9886, -1.1453],
                       [-0.2568, -0.1171,  0.4791,  0.4327],
                       [ 0.2909, -0.2474,  0.5359,  0.8001]],
         

In [26]:
%timeit Batch.stack(batches)

96.5 µs ± 440 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.9584,  1.1344,  0.1895],
               [-0.4369, -2.1558, -1.7903],
               [-0.9584,  1.1344,  0.1895],
               [-0.4369, -2.1558, -1.7903],
               [-0.9584,  1.1344,  0.1895],
               [-0.4369, -2.1558, -1.7903],
               [-0.9584,  1.1344,  0.1895],
               [-0.4369, -2.1558, -1.7903],
               [-0.9584,  1.1344,  0.1895],
               [-0.4369, -2.1558, -1.7903],
               [-0.9584,  1.1344,  0.1895],
               [-0.4369, -2.1558, -1.7903],
               [-0.9584,  1.1344,  0.1895],
               [-0.4369, -2.1558, -1.7903],
               [-0.9584,  1.1344,  0.1895],
               [-0.4369, -2.1558, -1.7903]]),
    x: Batch(
           c: tensor([[-0.8380, -0.9573,  0.9886, -1.1453],
                      [-0.2568, -0.1171,  0.4791,  0.4327],
                      [ 0.2909, -0.2474,  0.5359,  0.8001],
                      [-0.8380, -0.9573,  0.9886, -1.1453],
                      [-0.2568, -

In [28]:
%timeit Batch.cat(batches)

179 µs ± 1.23 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

415 µs ± 4.86 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
